In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
titanic_df = pd.read_csv('/content/drive/MyDrive/study/data/titanic.csv')
titanic_df.head()

,Unnamed: 0,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
titanic_filter = titanic_df[['Sex', 'Embarked']]
titanic_filter.head()

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S


In [7]:
one_hot = pd.get_dummies(titanic_filter)
one_hot.head()

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,False,True,False,False,True
1,True,False,True,False,False
2,True,False,False,False,True
3,True,False,False,False,True
4,False,True,False,False,True


In [8]:
one_hot_2 = pd.get_dummies(data=titanic_df, columns=['Sex', 'Embarked'])
one_hot_2.head()

,Unnamed: 0,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,False,True,False,False,True
1,1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,True,False,True,False,False
2,2,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,True,False,False,False,True
3,3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,True,False,False,False,True
4,4,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,False,True,False,False,True


## 정규화

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# ✅ 에러 없이 동작하는 최신 방식 (이모지 주석 포함)
from sklearn.preprocessing import PolynomialFeatures
from sklearn import __version__ as skl_version

from math import sqrt

import numpy as np
import pandas as pd

In [14]:
print("scikit-learn version:", skl_version)   # 🧪 버전 확인(참고용)

scikit-learn version: 1.6.1


In [10]:
admission_df = pd.read_csv('/content/drive/MyDrive/study/data/admission_data.csv').drop('Serial No.', axis=1)
admission_df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


GPT 추천(최신번전)

In [15]:
# 1) 타깃(y) 제거하고 X 준비
X = admission_df.drop(columns=['Chance of Admit '])  # 🎯 입력 특성만 남겨요

# 2) 다항 변환기 준비
pf = PolynomialFeatures(
    degree=6,          # 🔢 6차 다항식
    include_bias=False # 🧱 1(상수항) 컬럼은 빼요(중복/누적 위험↓)
    # interaction_only=True  # ✂️ 교호작용만 원하면 이 옵션 고려
)

# 3) 학습+변환 (NumPy 배열로 변환해도 되고 DataFrame 그대로 넣어도 OK)
X_poly = pf.fit_transform(X.to_numpy())  # ⚙️ (샘플, 다항특성수)로 변환

# 4) 새 컬럼명 뽑기(버전에 따라 안전하게 처리)
try:
    feature_names = pf.get_feature_names_out(X.columns)  # ✅ 최신 권장 API
except AttributeError:
    # ⬇️ 구버전 호환 (0.x~1.0 미만)
    feature_names = pf.get_feature_names(X.columns)

# 5) DataFrame으로 깔끔하게 정리(인덱스 유지)
X_poly_df = pd.DataFrame(X_poly, columns=feature_names, index=X.index)

# 6) 크기/메모리 확인(6차는 매우 큼!)
print("입력 X shape:", X.shape, "→ 다항 X shape:", X_poly_df.shape)  # 📏 크기 비교
print("대략 메모리(추정): ~", X_poly_df.memory_usage().sum()/1e6, "MB") # 💾 메모리 참고

X_poly_df.head()  # 👀 미리보기

입력 X shape: (500, 7) → 다항 X shape: (500, 1715)
대략 메모리(추정): ~ 6.860132 MB


,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,GRE Score^2,GRE Score TOEFL Score,GRE Score University Rating,...,LOR CGPA^2 Research^3,LOR CGPA Research^4,LOR Research^5,CGPA^6,CGPA^5 Research,CGPA^4 Research^2,CGPA^3 Research^3,CGPA^2 Research^4,CGPA Research^5,Research^6
0,337.0,118.0,4.0,4.5,4.5,9.65,1.0,113569.0,39766.0,1348.0,...,419.05125,43.425,4.5,807539.696082,83682.870060,8671.800006,898.632125,93.1225,9.65,1.0
1,324.0,107.0,4.0,4.0,4.5,8.87,1.0,104976.0,34668.0,1296.0,...,354.04605,39.915,4.5,487014.306256,54905.784245,6190.054594,697.864103,78.6769,8.87,1.0
2,316.0,104.0,3.0,3.0,3.5,8.00,1.0,99856.0,32864.0,948.0,...,224.00000,28.000,3.5,262144.000000,32768.000000,4096.000000,512.000000,64.0000,8.00,1.0
3,322.0,110.0,3.0,3.5,2.5,8.67,1.0,103684.0,35420.0,966.0,...,187.92225,21.675,2.5,424731.610940,48988.651781,5650.363527,651.714363,75.1689,8.67,1.0
4,314.0,103.0,2.0,2.0,3.0,8.21,0.0,98596.0,32342.0,628.0,...,0.00000,0.000,0.0,306237.903347,0.000000,0.000000,0.000000,0.0000,0.00,0.0


codeit 추천코드

In [17]:
X = admission_df.drop(columns=['Chance of Admit '])  # 🎯 타깃(y) 컬럼만 빼고 입력(X)만 남겨요

polynomial_transformer = PolynomialFeatures(6)        # 🔢 6차 다항식 변환기 생성 (기본값: include_bias=True)
polynomial_features = polynomial_transformer.fit_transform(X.values)
# ⚙️ X의 값을 NumPy 배열로 바꿔서(Fit+Transform) 다항특성으로 변환해요
#    결과는 (샘플수, 생성된_다항특성수) 모양의 큰 행렬이 돼요

features = polynomial_transformer.get_feature_names_out(X.columns)
# 🏷️ 원래 X의 컬럼명(X.columns)을 넘겨서, 새로 생긴 다항특성들의 이름을 만들어 받아요
#    예: GRE^2, TOEFL*CGPA 처럼 조합/거듭제곱 이름이 자동 생성돼요

X = pd.DataFrame(polynomial_features, columns=features)  # 🗂️ 다항특성을 가진 새 DataFrame으로 재구성
X.head()                                                 # 👀 앞부분 미리보기


,1,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,GRE Score^2,GRE Score TOEFL Score,...,LOR CGPA^2 Research^3,LOR CGPA Research^4,LOR Research^5,CGPA^6,CGPA^5 Research,CGPA^4 Research^2,CGPA^3 Research^3,CGPA^2 Research^4,CGPA Research^5,Research^6
0,1.0,337.0,118.0,4.0,4.5,4.5,9.65,1.0,113569.0,39766.0,...,419.05125,43.425,4.5,807539.696082,83682.870060,8671.800006,898.632125,93.1225,9.65,1.0
1,1.0,324.0,107.0,4.0,4.0,4.5,8.87,1.0,104976.0,34668.0,...,354.04605,39.915,4.5,487014.306256,54905.784245,6190.054594,697.864103,78.6769,8.87,1.0
2,1.0,316.0,104.0,3.0,3.0,3.5,8.00,1.0,99856.0,32864.0,...,224.00000,28.000,3.5,262144.000000,32768.000000,4096.000000,512.000000,64.0000,8.00,1.0
3,1.0,322.0,110.0,3.0,3.5,2.5,8.67,1.0,103684.0,35420.0,...,187.92225,21.675,2.5,424731.610940,48988.651781,5650.363527,651.714363,75.1689,8.67,1.0
4,1.0,314.0,103.0,2.0,2.0,3.0,8.21,0.0,98596.0,32342.0,...,0.00000,0.000,0.0,306237.903347,0.000000,0.000000,0.000000,0.0000,0.00,0.0


| 항목             | Codeit               | ChatGPT 코드          | 차이가 왜 중요한가                                                                                                                                                                                                       |
| -------------- | ------------------ | ------------------ | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `include_bias` | 기본값(생략) → **True** | **False** 권장       | `include_bias=True`면 \*\*상수항(1 열)\*\*이 추가돼요.  
|    |    |   |그런데 `LinearRegression` 기본이
|    |    |   |`fit_intercept=True`라서 **상수항이 2개**가 됩니다(중복/다중공선성 ↑).
|    |    |   |→ 해결: `include_bias=False` **또는** `LinearRegression(fit_intercept=False)`
|    |    |   ||중 하나만 쓰기. |
| 입력 형태          | `X.values`         | `X.to_numpy()`     | 둘 다 NumPy 배열을 얻지만, `to_numpy()`가 **명시적**이고 옵션(`dtype`, `copy`) 제어가 쉬워요. 기능상 차이는 거의 없음.                                                                                                                           |
| 변수 이름          | `X`를 덮어씀           | `X_poly_df` 등 새 이름 | 같은 이름을 덮어쓰면 디버깅이 헷갈려요. 원본 X를 남기고 `X_poly_df`로 새 DF를 만드는 걸 추천.                                                                                                                                                    |
| 인덱스 유지         | (미지정)              | `index=X.index` 명시 | 원본 인덱스를 유지하면 이후 **y와의 정렬**에서 안전해요.                                                                                                                                                                               |
| 차수 6           | 동일                 | 동일                 | 6차는 특성 수가 **폭증**합니다. 메모리/시간 점검 필수(아래 팁 참고).                                                                                                                                                                      |


In [22]:
y = admission_df['Chance of Admit ']
y

,Chance of Admit
0,0.92
1,0.76
2,0.72
3,0.80
4,0.65
...,...
495,0.87
496,0.96
497,0.93
498,0.73


In [31]:
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.3, random_state=5)

In [32]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [33]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [34]:
mse = mean_squared_error(y_train, y_train_pred)

print('Training set에서의 성능')
print('--------------------')
print(sqrt(mse))

mse = mean_squared_error(y_test, y_test_pred)

print('Test set에서의 성능')
print('--------------------')
print(sqrt(mse))

Training set에서의 성능
--------------------
0.020373026858386902
Test set에서의 성능
--------------------
1.1320864381296558


In [37]:
# L1정규화 Lasso모델
from sklearn.linear_model import Lasso

model_la = Lasso(alpha=0.001, max_iter=1000)
model_la.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.132e-01, tolerance: 7.568e-04
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.001)

In [39]:
y_train_pred_la = model_la.predict(X_train)
y_test_pred_la = model_la.predict(X_test)

In [41]:
mse = mean_squared_error(y_train, y_train_pred)

print('LASSO_Training set에서의 성능')
print('--------------------')
print(sqrt(mse))

mse = mean_squared_error(y_test, y_test_pred)

print('LASSO_Test set에서의 성능')
print('--------------------')
print(sqrt(mse))

LASSO_Training set에서의 성능
--------------------
0.05389371888907575
LASSO_Test set에서의 성능
--------------------
0.06689735278061708


In [45]:
# L2정규화 RIDGE모델
from sklearn.linear_model import Ridge

model_rid = Ridge(alpha=0.001, max_iter=1000)
model_rid.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Ridge(alpha=0.001, max_iter=1000)

In [46]:
y_train_pred_rid = model_rid.predict(X_train)
y_test_pred_rid = model_rid.predict(X_test)

In [47]:
mse = mean_squared_error(y_train, y_train_pred)

print('RIDGE_Training set에서의 성능')
print('--------------------')
print(sqrt(mse))

mse = mean_squared_error(y_test, y_test_pred)

print('RIDGE_Test set에서의 성능')
print('--------------------')
print(sqrt(mse))

RIDGE_Training set에서의 성능
--------------------
0.05389371888907575
RIDGE_Test set에서의 성능
--------------------
0.06689735278061708


In [54]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import numpy as np
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [55]:
iris_data = datasets.load_iris()

In [56]:
X = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
y = pd.DataFrame(iris_data.target, columns=['Class'])

In [57]:
logistic_model = LogisticRegression(max_iter=2000)

In [61]:
np.average(cross_val_score(logistic_model, X, y.values.ravel(), cv=5))

np.float64(0.9733333333333334)

### scikit-learn으로 그리드 서치해보기


In [67]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# ✅ 에러 없이 동작하는 최신 방식 (이모지 주석 포함)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn import __version__ as skl_version

from math import sqrt

import numpy as np
import pandas as pd

In [68]:
print("scikit-learn version:", skl_version)   # 🧪 버전 확인(참고용)

scikit-learn version: 1.6.1


In [69]:
admission_df = pd.read_csv('/content/drive/MyDrive/study/data/admission_data.csv').drop('Serial No.', axis=1)
admission_df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [73]:
X = admission_df.drop(columns=['Chance of Admit '])  # 🎯 타깃(y) 컬럼만 빼고 입력(X)만 남겨요

polynomial_transformer = PolynomialFeatures(2)        # 🔢 6차 다항식 변환기 생성 (기본값: include_bias=True)
polynomial_features = polynomial_transformer.fit_transform(X.values)
# ⚙️ X의 값을 NumPy 배열로 바꿔서(Fit+Transform) 다항특성으로 변환해요
#    결과는 (샘플수, 생성된_다항특성수) 모양의 큰 행렬이 돼요

features = polynomial_transformer.get_feature_names_out(X.columns)
# 🏷️ 원래 X의 컬럼명(X.columns)을 넘겨서, 새로 생긴 다항특성들의 이름을 만들어 받아요
#    예: GRE^2, TOEFL*CGPA 처럼 조합/거듭제곱 이름이 자동 생성돼요

X = pd.DataFrame(polynomial_features, columns=features)  # 🗂️ 다항특성을 가진 새 DataFrame으로 재구성
X.head()                                                 # 👀 앞부분 미리보기


,1,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,GRE Score^2,GRE Score TOEFL Score,...,SOP^2,SOP LOR,SOP CGPA,SOP Research,LOR ^2,LOR CGPA,LOR Research,CGPA^2,CGPA Research,Research^2
0,1.0,337.0,118.0,4.0,4.5,4.5,9.65,1.0,113569.0,39766.0,...,20.25,20.25,43.425,4.5,20.25,43.425,4.5,93.1225,9.65,1.0
1,1.0,324.0,107.0,4.0,4.0,4.5,8.87,1.0,104976.0,34668.0,...,16.00,18.00,35.480,4.0,20.25,39.915,4.5,78.6769,8.87,1.0
2,1.0,316.0,104.0,3.0,3.0,3.5,8.00,1.0,99856.0,32864.0,...,9.00,10.50,24.000,3.0,12.25,28.000,3.5,64.0000,8.00,1.0
3,1.0,322.0,110.0,3.0,3.5,2.5,8.67,1.0,103684.0,35420.0,...,12.25,8.75,30.345,3.5,6.25,21.675,2.5,75.1689,8.67,1.0
4,1.0,314.0,103.0,2.0,2.0,3.0,8.21,0.0,98596.0,32342.0,...,4.00,6.00,16.420,0.0,9.00,24.630,0.0,67.4041,0.00,0.0


In [71]:
y = admission_df['Chance of Admit ']
y

,Chance of Admit
0,0.92
1,0.76
2,0.72
3,0.80
4,0.65
...,...
495,0.87
496,0.96
497,0.93
498,0.73


In [74]:
hyper_parameter= {
    'alpha': [0.01, 0.1, 1, 10],
    'max_iter': [100, 500, 1000, 1500, 2000]
}

In [75]:
lasso_model = Lasso()

In [76]:
hyper_parameter_tuner = GridSearchCV(lasso_model, hyper_parameter, cv=5)
hyper_parameter_tuner.fit(X, y)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.253e-01, tolerance: 6.706e-04
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.086e-01, tolerance: 8.246e-04
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.684e-01, tolerance: 8.590e

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.01, 0.1, 1, 10],
                         'max_iter': [100, 500, 1000, 1500, 2000]})

In [77]:
hyper_parameter_tuner.best_params_

{'alpha': 0.1, 'max_iter': 500}